**Корректность проверена на Python 3.6:**
+ gensim 4.0.1

# Пример использования библиотеки gensim для тематического моделирования
## Данная модель основана на байесовских методов машинного обучения

In [100]:
from gensim import corpora, models
import re

In [101]:
# Импортируем данные в формте UCI Bag of Words
data = corpora.UciCorpus("docword.xkcd.uci", "vocab.xkcd.uci")
dictionary = data.create_dictionary()

In [102]:
# обучение модель
num_topics = 2
%time ldamodel = models.ldamodel.LdaModel(data, id2word=dictionary, num_topics=num_topics, passes=20, alpha=1.25, eta=1.25)

Wall time: 194 ms


In [103]:
# Сохранение модели
ldamodel.save("ldamodel_xkcd")

In [104]:
# Загрузка модели
ldamodel = models.ldamodel.LdaModel.load("ldamodel_xkcd")

In [105]:
# выводим топы слов
num_words = 3
for t, top_words in ldamodel.print_topics(num_topics=2, num_words=num_words):
    print("Topic", t, ":", top_words)

Topic 0 : 0.181*"b'industry'" + 0.114*"b'distance'" + 0.097*"b'co-financing'"
Topic 1 : 0.257*"b'subsidy'" + 0.103*"b'loan'" + 0.068*"b'support'"


In [106]:
# Вычисляем логарифм перплексии и немного преобразуем, чтобы привести к общепринятому виду
perplexity = ldamodel.log_perplexity(list(data))
print(2**(-perplexity))

6.658206345718724


In [107]:
sum_words = 150
perp = ldamodel.bound(data)
2**(-perp/float(sum_words))

2.927986787128405

In [121]:
# Получение распределения тем для конкретного документа
# Индекс + 1 списка data соответствует id меры поддержки (нумерация мер с 1 до последней меры)
distrib_theme = {}
doc = list(data)
for i in range(len(list(data))):
    distrib_theme.setdefault(i, 0)
    if ldamodel.get_document_topics(doc)[i][0][1] < ldamodel.get_document_topics(doc)[i][1][1]:
        distrib_theme[i] = 1
    else:
        distrib_theme[i] = 0



In [109]:
# Получение топовых слов по теме
# Индекс списка соответствует тематике
count_of_themes = 6
words_of_theme = []
for j in range(num_topics):
    df = [re.split('[^a-z]', sentence.lower().replace('\n', '')) for sentence in \
         ldamodel.print_topics(num_topics=num_topics, num_words=3)[j][1].split('+')]
    words_of_theme.append(list(set([word for sen in df for word in sen if len(word) > 2])))


### Получение запроса в текстовом формате

In [110]:
text_info = 'Subsidy finacial support'
data_list = text_info.split()
data_list_lower = []
for word in data_list:
    lw = word.lower().replace('\n', '')
    data_list_lower.append(lw)
words_user = list(set(data_list_lower))

In [113]:
list_of_themes = []
for i in range(len(words_user)):
    for j in range(len(words_of_theme)):
        for y in range(num_words):
            if words_user[i] == words_of_theme[j][y]:
                if ldamodel.get_document_topics(doc)[0][1] > ldamodel.get_document_topics(doc)[1][1]:
                    list_of_themes.append(ldamodel.get_document_topics(doc)[0])
                else:
                    list_of_themes.append(ldamodel.get_document_topics(doc)[1])


In [134]:
themes = []
for j, v in distrib_theme.items():
    for i in range(len(list_of_themes)):
        if v == list_of_themes[i][0]:
            themes.append(j+1)
        

In [135]:
themes = set(themes)

In [136]:
themes

{1, 2, 3, 4}